In [ ]:
# Reddit Data Import Script that fetches (hopefully) data from subreddits and stores in Oracle

import os
import sys
from datetime import datetime
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("Reddit Data Import Script aka jdem na to kamo")

# this checks for required dependencies and import from utils
try:
    from utils import get_oracle_connection, fetch_and_insert_posts_comments, safe_execute
    print("Utils functions imported - so far so good")
except ImportError as e:
    print(f"Error importing from utils - upsis: {e}")
    sys.exit(1)

try:
    import praw
    print("PRAW (Reddit API) rdy nice :)")
except ImportError:
    print("PRAW not available. jeste nejsme cooked -> pip install praw")
    sys.exit(1)

try:
    import oracledb
    print("Oracle DB available letzgoo")
except ImportError:
    print("Oracle DB not available. Ale jeste nejsme cooked -> pip install oracledb")
    sys.exit(1)

# Reddit API setup
try:
    reddit = praw.Reddit(
        client_id=os.getenv("api-client_id"),
        client_secret=os.getenv("api-client_secret"),
        user_agent="stock_sentiment_scraper_v2.0",
        username=os.getenv("api-username"),
        password=os.getenv("api-password")
    )
    
    # Test Reddit connection
    print(f"Reddit API connected as: {reddit.user.me()}")
    
except Exception as e:
    print(f"Reddit API connection failed: {e}")
    sys.exit(1)

# Database connection and data import
conn = get_oracle_connection()
if conn:
    cursor = conn.cursor()
    print("Database connection successful - SIUUU")
    
    # subreddits 
    try:
        cursor.execute("SELECT subreddit, last_fetched_utc FROM subreddit_tracker")
        subreddit_data = cursor.fetchall()
        print(f"Found {len(subreddit_data)} subreddits to process")
        
        # Process each subreddit
        for sub, last_utc in subreddit_data:
            print(f"\nFetching new data for subreddit: r/{sub}")
            print(f"   Last fetched UTC: {last_utc}")
            
            # inserting
            conn, cursor, max_utc = fetch_and_insert_posts_comments(
                reddit, sub, last_fetched_utc=last_utc, conn=conn, cursor=cursor
            )
            
            # Updating the tracker if new data
            if max_utc > last_utc:
                conn, cursor = safe_execute(conn, cursor, """
                    UPDATE subreddit_tracker SET last_fetched_utc = :max_utc WHERE subreddit = :sub
                """, {"max_utc": max_utc, "sub": sub})
                print(f"Updated last_fetched_utc for r/{sub} to {max_utc}")
            else:
                print(f"No new data found for r/{sub}")
        
        # Close connections
        cursor.close()
        conn.close()
        print("\nData import completed successfully. Letzgoo!")
        print("Database connection closed")
        
    except Exception as e:
        print(f"Error during data import (fuck): {e}")
        cursor.close()
        conn.close()
        
else:
    print("Database connection failed - well fuck")
    sys.exit(1)